## Project 1: Analytics on Glassdoor Reviews and Yelp Category Data

### University of California, Santa Barbara  
### PSTAT 135/235  
### Last Updated: Jan 12, 2019
### Kevin Lorenzo Ayala, 4871505

### OBJECTIVE  
#### In this assignment, you will perform some basic analytics on review and category data.
#### This will entail performing operations on *RDDs*, and using *list comprehensions*.
#### Read in the dataset and perform the steps requested below.

#### TOTAL POINTS = 10

### Config Setup

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("review_and_category_analytics") \
    .config("spark.executor.memory", '8g') \
    .config('spark.executor.cores', '4') \
    .config('spark.cores.max', '4') \
    .config("spark.driver.memory",'8g') \
    .getOrCreate()

sc = spark.sparkContext

Read in the dataset  

In [3]:
data_pathway = sc.textFile("data/reviews_and_categories.csv")

In [4]:
data_pathway.take(5)

['index,review_emp_txt,categories',
 '0,,"[\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"',
 '1,,[]',
 '2,,"[\'other\', \'food & beverages\']"',
 '3,"Some franchise owners dock hours. Pros Good discounts on the food. Cons The location where I was working, in North Fresno near Riverpark Mall, was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff, in front of the guests, if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks, and ask employees to stay an hour or two past the end of their shift, but would not pay them for their time.","[\'lunch\', \'best sandwich\', \'entertainment\', \'restaurants\', \'sub\', \'arizona\', \'quick\', \'social networks\', \'washington\', \'catering reno\', \'establishment

In [5]:
header = data_pathway.first()

In [6]:
header

'index,review_emp_txt,categories'

get non-header records

In [8]:

newdata_pathway = data_pathway.filter(lambda x: x != header)\
        .map(lambda row: [i for i in row.split(',')])


print the first 2 records (note: exclude the header in all calculations)

In [9]:
newdata_pathway.take(2)

[['0',
  '',
  '"[\'point of interest\'',
  " 'mexican'",
  " 'establishment'",
  " 'food'",
  ' \'restaurant\']"'],
 ['1', '', '[]']]

**1) get a record count (2 POINTS)**

In [10]:
newdata_pathway.count()

1305

store records with non-empty *review_emp_txt*

In [72]:
non_missing_reviews = newdata_pathway.map(lambda x: (x[0],x[1],x[2:])).filter(lambda x: x[1] != '')

**2) get a count of records with non-missing reviews (2 POINTS)**

In [73]:
non_missing_reviews.count()

305

In [74]:
non_missing_reviews.take(5)

[('3',
  '"Some franchise owners dock hours. Pros Good discounts on the food. Cons The location where I was working',
  [' in North Fresno near Riverpark Mall',
   ' was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff',
   ' in front of the guests',
   ' if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks',
   ' and ask employees to stay an hour or two past the end of their shift',
   ' but would not pay them for their time."',
   '"[\'lunch\'',
   " 'best sandwich'",
   " 'entertainment'",
   " 'restaurants'",
   " 'sub'",
   " 'arizona'",
   " 'quick'",
   " 'social networks'",
   " 'washington'",
   " 'catering reno'",
   " 'establishment'",
   " 'nevada'",
   " 'restaurant'",
   " 'wraps'",
   " 'qsr'",
   " 'small business'",

**3) Return the count of records where review contains the word *awesome*  (2 POINTS)**

In [70]:
awesome_counts = non_missing_reviews.map(lambda x: x[1]).filter(lambda x: 'awesome' in x) 

In [71]:
awesome_counts.count()

3

Print the records where review contains the word *awesome*

In [48]:
awesome_counts.collect()

['Very awesome. Pros They allow the use of flexible schedule. Cons There are too many hostesses.',
 'Not a bad place to work Pros The couple that owned the one I worked at were very big on school and very flexible with my school schedule Being a delivery driver is great because the tips are awesome Cons The owner was kind of crazy I m kind of a health freak and just don t like working with pizza',
 '"Cashier Play Attendant Pros The kids are awesome it feels like a family everyone is willing to help awesome food discount Cons needs new equipment like a new espresso machine']

4) Lowercase all reviews, then return the count of records where review contains the word *awesome*

In [81]:
 awesome_lowered = non_missing_reviews.map(lambda x: x[1].lower()).filter(lambda x: 'awesome' in x)

In [104]:
awesome_lowered.count()

5

In [105]:
awesome_lowered.collect()

['very awesome. pros they allow the use of flexible schedule. cons there are too many hostesses.',
 'not a bad place to work pros the couple that owned the one i worked at were very big on school and very flexible with my school schedule being a delivery driver is great because the tips are awesome cons the owner was kind of crazy i m kind of a health freak and just don t like working with pizza',
 '"cashier play attendant pros the kids are awesome it feels like a family everyone is willing to help awesome food discount cons needs new equipment like a new espresso machine',
 'busboy pros flexible schedule decent tips awesome coworkers 50 employee discount on meals cons physically demanding long hours rude and or impatient customers',
 '"awesome place to work pros management is laid back and the work environment is relaxed. everyone that works there is willing to help when you need anything. lots of college kids work there because it is a college town. it s a tremendous job for anyone i

**4) Return the top 10 most frequent categories  (4 POINTS)**  

Preprocess the categories by:  
* stripping characters: &nbsp; [ &nbsp; ] &nbsp;  ' &nbsp;  "  
* trim spaces before and after words  
* lowercase

NOTE: Be sure to keep terms together, for example 'jet skiing' should not become 'jet', 'skiing'

In [115]:
yelp_categories = newdata_pathway.map(lambda x: x[2:])

In [116]:
yelp_categories.take(10)

[['"[\'point of interest\'',
  " 'mexican'",
  " 'establishment'",
  " 'food'",
  ' \'restaurant\']"'],
 ['[]'],
 ['"[\'other\'', ' \'food & beverages\']"'],
 [' in North Fresno near Riverpark Mall',
  ' was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff',
  ' in front of the guests',
  ' if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks',
  ' and ask employees to stay an hour or two past the end of their shift',
  ' but would not pay them for their time."',
  '"[\'lunch\'',
  " 'best sandwich'",
  " 'entertainment'",
  " 'restaurants'",
  " 'sub'",
  " 'arizona'",
  " 'quick'",
  " 'social networks'",
  " 'washington'",
  " 'catering reno'",
  " 'establishment'",
  " 'nevada'",
  " 'restaurant'",
  " 'wraps'",
  " 'qsr'",
  " 

In [145]:
yelp_categories_2 = yelp_categories.map(lambda row: [token.replace('[','').replace(']','').replace("'",'') .replace('"','')
.strip().lower() for token in row]).flatMap(lambda x: x).map(lambda x: (x,1)) \
.reduceByKey(lambda x,y:x+y).map(lambda x:(x[1],x[0])).sortByKey(False)

                                 
                                                        
                                        

In [143]:
yelp_categories_2.collect()

[(718, 'point of interest'),
 (718, 'establishment'),
 (717, 'food'),
 (660, 'restaurant'),
 (497, 'price'),
 (483, 'other'),
 (439, ''),
 (332, 'credit cards'),
 (311, 'menus'),
 (292, 'eating places'),
 (274, 'dining options'),
 (231, 'restaurants'),
 (211, '$$'),
 (197, 'no outdoor seating'),
 (171, 'dinner'),
 (166, 'no reservations'),
 (150, 'outdoor seating'),
 (129, 'lunch & more'),
 (123, 'mexican'),
 (123, 'drinks'),
 (101, 'mexican restaurant'),
 (91, 'hospitality'),
 (85, 'sandwiches'),
 (84, 'fast food'),
 (82, 'bar'),
 (81, 'reservations'),
 (80, 'store'),
 (74, 'american restaurant'),
 (74, 'delivery'),
 (73, 'happy hour'),
 (72, 'beer'),
 (68, 'meal takeaway'),
 (68, 'california'),
 (66, 'restaurant chains'),
 (66, 'wi-fi'),
 (58, 'pizza'),
 (54, 'wine'),
 (53, 'american (traditional)'),
 (49, 'cafe'),
 (47, 'full bar & cocktails'),
 (45, 'lunch'),
 (45, 'chinese'),
 (44, 'seafood'),
 (44, 'brunch & more'),
 (43, 'burgers'),
 (42, 'italian'),
 (41, 'pizza place'),
 (41, 